In [ ]:
# 08_Regression_1.ipynb

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("taxi-fare-predicition").getOrCreate()

In [2]:
import os
cwd = os.getcwd()
trip_data_path = os.path.join(cwd, 'learning_spark_data', 'trips', '*.csv')
trip_data_path

'/home/jovyan/work/learning_spark_data/trips/*.csv'

In [4]:
file_path = f"file:///{trip_data_path.replace(os.sep,'/')}"
file_path

'file:////home/jovyan/work/learning_spark_data/trips/*.csv'

In [ ]:
trip_df = spark.read.csv(file_path, inferSchema=True, header=True)
trip_df.printSchema()

In [ ]:
# 운행거리에 따른 요금 예측

In [8]:
trip_df.createOrReplaceTempView('trips')

In [9]:
query = """
SELECT
    trip_distance,
    total_amount
FROM trips

WHERE total_amount < 5000
  AND total_amount > 0
  AND trip_distance > 0
  AND trip_distance < 500
  AND passenger_count < 4
  AND TO_DATE(tpep_pickup_datetime) >= "2021-01-01"
  AND TO_DATE(tpep_pickup_datetime) < "2021-08-01"
"""

In [10]:
trip_df = spark.sql(query)
trip_df.createOrReplaceTempView('data')

In [13]:
# data table 결과 확인
spark.sql('select * from data limit 5').show(3)

+-------------+------------+
|trip_distance|total_amount|
+-------------+------------+
|         16.5|       70.07|
|         1.13|       11.16|
|         2.68|       18.59|
+-------------+------------+
only showing top 3 rows



In [17]:
# train, test split 8:2, seed =1
train_data, test_data = trip_df.randomSplit([0.8,0.2], seed=1)

In [19]:
# vectorassembler > feature : trip_distance, target: total_amount
from pyspark.ml.feature import VectorAssembler
vassembler = VectorAssembler(inputCols=['trip_distance'], outputCol='features')
vtrain_df = vassembler.transform(train_data)
vtrain_df.select('features', 'total_amount').show(5)

+--------+------------+
|features|total_amount|
+--------+------------+
|  [0.01]|        3.05|
|  [0.01]|         3.3|
|  [0.01]|         3.3|
|  [0.01]|         3.3|
|  [0.01]|         3.3|
+--------+------------+
only showing top 5 rows



In [22]:
# LinearRegression 생성 maxIter=50, LabelCol='total_amount', featuresCol='features'
# fit
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(maxIter=50, labelCol='total_amount', featuresCol='features')
lr_model = lr.fit(vtrain_df) #천만건데이터 학습

# vassembler, transform (test)
# model.transform()
vtest_df = vassembler.transform(test_data)
pred = lr_model.transform(vtest_df) #2백만건데이터 예측

In [23]:
pred.show(5)

+-------------+------------+--------+-----------------+
|trip_distance|total_amount|features|       prediction|
+-------------+------------+--------+-----------------+
|         0.01|         3.3|  [0.01]|9.430440745312902|
|         0.01|         3.3|  [0.01]|9.430440745312902|
|         0.01|         3.3|  [0.01]|9.430440745312902|
|         0.01|         3.3|  [0.01]|9.430440745312902|
|         0.01|         3.3|  [0.01]|9.430440745312902|
+-------------+------------+--------+-----------------+
only showing top 5 rows



In [24]:
lr_model.summary.rootMeanSquaredError #RMSE

6.30781413196623

In [25]:
lr_model.summary.r2

0.7648633777017714

In [ ]:
# 새로운 데이터로 예측하기

In [29]:
from pyspark.sql.types import DoubleType

new_distance_list = [1.1, 5.4, 10.2, 30.0]
distance_df = spark.createDataFrame(new_distance_list, DoubleType()).toDF('trip_distance')
distance_df.show()

+-------------+
|trip_distance|
+-------------+
|          1.1|
|          5.4|
|         10.2|
|         30.0|
+-------------+



In [30]:
vdistance_df = vassembler.transform(distance_df)
lr_model.transform(vdistance_df).show()

+-------------+--------+------------------+
|trip_distance|features|        prediction|
+-------------+--------+------------------+
|          1.1|   [1.1]|12.672809485363317|
|          5.4|   [5.4]|25.463805432351194|
|         10.2|  [10.2]| 39.74212648945393|
|         30.0|  [30.0]| 98.64020085000274|
+-------------+--------+------------------+



In [31]:
spark.stop()